In [1]:
!pip install -q transformers datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.0 MB/s eta 0:00:00


In [2]:
!pip install -q transformers[torch] -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 9.3 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from transformers import DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
dataset = load_dataset("imdb", split={'train': 'train[:5%]', 'validation': 'test[:1%]'})

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [4]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
training_args = TrainingArguments(
    output_dir='/content/results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='/content/logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="no",
    report_to="none",
    load_best_model_at_end=False,
)


In [6]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()


Epoch,Training Loss,Validation Loss
1,0.001100,0.000493


TrainOutput(global_step=79, training_loss=0.11039629120591886, metrics={'train_runtime': 124.0863, 'train_samples_per_second': 10.074, 'train_steps_per_second': 0.637, 'total_flos': 328115930474880.0, 'train_loss': 0.11039629120591886, 'epoch': 1.0})

In [9]:
from transformers import pipeline

# Load the tokenizer and model (before fine-tuning)
pretrained_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
pretrained_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

# Create a pipeline for sentiment analysis
pretrained_pipeline = pipeline("text-classification", model=pretrained_model, tokenizer=pretrained_tokenizer)

# Example inputs
examples = [
    "I loved the new Batman movie!",
    "I did not enjoy my last vacation.",
    "The food at the restaurant was great!",
    "The weather today is depressing."
]

# Get predictions (before fine-tuning)
pretrained_predictions = pretrained_pipeline(examples)

# Display predictions
for example, prediction in zip(examples, pretrained_predictions):
    print(f"Input: {example}\nPre-trained model prediction: {prediction}\n")



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Input: I loved the new Batman movie!
Pre-trained model prediction: {'label': 'LABEL_1', 'score': 0.5877944231033325}

Input: I did not enjoy my last vacation.
Pre-trained model prediction: {'label': 'LABEL_1', 'score': 0.5768372416496277}

Input: The food at the restaurant was great!
Pre-trained model prediction: {'label': 'LABEL_1', 'score': 0.5941697955131531}

Input: The weather today is depressing.
Pre-trained model prediction: {'label': 'LABEL_1', 'score': 0.5794910192489624}



In [ ]:
# After training, save the model manually
model.save_pretrained("/content/results")
tokenizer.save_pretrained("/content/results")

In [13]:
# Load the fine-tuned model and tokenizer
# Make sure to replace `model_checkpoint` with the path or name of your fine-tuned model
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("/content/results")
fine_tuned_pipeline = pipeline("text-classification", model=fine_tuned_model, tokenizer=tokenizer)

# Get predictions (after fine-tuning)
fine_tuned_predictions = fine_tuned_pipeline(examples)

# Display new predictions
for example, prediction in zip(examples, fine_tuned_predictions):
    print(f"Input: {example}\nFine-tuned model prediction: {prediction}\n")


Input: I loved the new Batman movie!
Fine-tuned model prediction: {'label': 'LABEL_0', 'score': 0.9983172416687012}

Input: I did not enjoy my last vacation.
Fine-tuned model prediction: {'label': 'LABEL_0', 'score': 0.9984527826309204}

Input: The food at the restaurant was great!
Fine-tuned model prediction: {'label': 'LABEL_0', 'score': 0.9913864135742188}

Input: The weather today is depressing.
Fine-tuned model prediction: {'label': 'LABEL_0', 'score': 0.9964613318443298}

